In [2]:
import json
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import xgboost as xgb

Using TensorFlow backend.


In [3]:
def parse(file):
    for l in open(file, 'r'):
        yield json.loads(l)

In [4]:
data = parse('news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json')

In [5]:
df = pd.DataFrame(data)

In [6]:
df.head()

,article_link,headline,is_sarcastic
0,https://www.theonion.com/thirtysomething-scien...,thirtysomething scientists unveil doomsday clo...,1
1,https://www.huffingtonpost.com/entry/donna-edw...,dem rep. totally nails why congress is falling...,0
2,https://www.huffingtonpost.com/entry/eat-your-...,eat your veggies: 9 deliciously different recipes,0
3,https://local.theonion.com/inclement-weather-p...,inclement weather prevents liar from getting t...,1
4,https://www.theonion.com/mother-comes-pretty-c...,mother comes pretty close to using word 'strea...,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 3 columns):
article_link    28619 non-null object
headline        28619 non-null object
is_sarcastic    28619 non-null int64
dtypes: int64(1), object(2)
memory usage: 670.8+ KB


In [8]:
df = df.drop(df.columns[0], axis=1)

In [9]:
df.head()

,headline,is_sarcastic
0,thirtysomething scientists unveil doomsday clo...,1
1,dem rep. totally nails why congress is falling...,0
2,eat your veggies: 9 deliciously different recipes,0
3,inclement weather prevents liar from getting t...,1
4,mother comes pretty close to using word 'strea...,1


In [10]:
target = df.is_sarcastic
df = df.headline

In [11]:
y = pd.get_dummies(target).values

In [12]:
df.head()

0    thirtysomething scientists unveil doomsday clo...
1    dem rep. totally nails why congress is falling...
2    eat your veggies: 9 deliciously different recipes
3    inclement weather prevents liar from getting t...
4    mother comes pretty close to using word 'strea...
Name: headline, dtype: object

In [13]:
tokenizer = text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(df))
list_tokenized_train = tokenizer.texts_to_sequences(df)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=100)

In [13]:
embedding_size = 128
input_ = Input(shape=(100,))
x = Embedding(30000, embedding_size)(input_)
x = Bidirectional(LSTM(25, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.5)(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='sigmoid')(x)

model = Model(inputs=input_, outputs=x)

W0909 21:10:49.920628 139955204712192 deprecation_wrapper.py:119] From /home/matthew/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0909 21:10:50.190515 139955204712192 deprecation_wrapper.py:119] From /home/matthew/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0909 21:10:50.244853 139955204712192 deprecation_wrapper.py:119] From /home/matthew/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0909 21:10:51.126390 139955204712192 deprecation_wrapper.py:119] From /home/matthew/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.place

In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

W0909 21:10:52.146632 139955204712192 deprecation_wrapper.py:119] From /home/matthew/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0909 21:10:52.186915 139955204712192 deprecation_wrapper.py:119] From /home/matthew/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0909 21:10:52.195091 139955204712192 deprecation.py:323] From /home/matthew/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 128)          3840000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 50)           30800     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 50)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
__________

In [16]:
checkpoints_path = 'weights_base.best.hdf5'
checkpoint = ModelCheckpoint(checkpoints_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=25)
callbacks = [checkpoint, early_stopping]

In [17]:
model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.3, callbacks=callbacks)

Train on 20033 samples, validate on 8586 samples
Epoch 1/2
20033/20033 [==============================] - 90s 5ms/step - loss: 0.4603 - acc: 0.7777 - val_loss: 0.3315 - val_acc: 0.8551

Epoch 00001: val_loss improved from inf to 0.33151, saving model to weights_base.best.hdf5
Epoch 2/2
20033/20033 [==============================] - 88s 4ms/step - loss: 0.2136 - acc: 0.9214 - val_loss: 0.3339 - val_acc: 0.8559

Epoch 00002: val_loss did not improve from 0.33151
